In [ ]:
from colossus.cosmology import cosmology
from colossus.lss import mass_function
import matplotlib.pyplot as plt
import numpy as np
import math

x_mass = np.logspace(8.0, 16, 100)

plt.rcParams.update({'font.size': 16})
plt.figure(figsize=(10,6))
cosmology.setCosmology('bolshoi')

# Let's compare to whats in the halo mass function file we are using
fn = "../halo_mass_function.dat"
data = np.loadtxt(fn)
plt.plot(data[:,0], data[:,0]*data[:,1], 'r--', label='GF Default')

for z in [0.0]:
#for z in [0.0, 0.33, 0.75, 1.25, 2.2, 3.2, 5.5, 10.0]:
#for z in [0.0, 0.1, 0.2]:
    age = cosmology.getCurrent().age(z)
    bf = 0.156
    mfunc = mass_function.massFunction(x_mass, z, mdef = '200m', model = 'tinker08', q_out='dndlnM') 
    #color = plt.cm.inferno(0.9 - (age / 13.8))
    color = plt.cm.inferno(z / 0.3)
    plt.plot(x_mass, mfunc, color=color, label=f'z={z:.3f} ({age:.1f} Gyr)')
    
plt.xscale('log')
plt.yscale('log')
#plt.ylim(1e-3, 1e0)
plt.xlabel('$M_h$ [$M_\odot$] (Bolshoi Cosmology)')
plt.ylabel('Number Density')
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.tight_layout()